In [3]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from folium.plugins import TimeSliderChoropleth
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pycirclize import Circos
import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data
from d3blocks import D3Blocks
import seaborn as sns
from shapely.geometry import Point



In [7]:
# Leggo il file di shapes
comuni_shape = gpd.read_file('../shapes/Com01012021_g_WGS84.shp',encoding='utf-8',crs="epsg:32632")[['COMUNE','geometry']]

# Leggo il file con i tempi per solo comuni
comuni_list = pd.read_csv('../origini/paths.csv')

# Stops
stops = pd.read_csv('../gtfsFiles/extractedFromMerged/stops.txt')
stop_times = pd.read_csv('../gtfsFiles/extractedFromMerged/stop_times.txt')

/tmp/ipykernel_59431/897358836.py:9: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv('../gtfsFiles/extractedFromMerged/stop_times.txt')


In [8]:
comuni_shape = comuni_shape.to_crs('EPSG:4326')


In [9]:
stops

,stop_id,stop_name,stop_lat,stop_lon,stop_code
0,1@FL0101_600,Marina Vecchia Itis Dir.Monti,44.028367,10.129152,1@FL0101
1,1@FL0102_600,Marina Vecchia Itis Dir.Mare,44.028527,10.129167,1@FL0102
2,1@FL0282_600,Ss1/Sede Alberghiero,44.040226,10.124439,1@FL0282
3,1@FL0471_600,Ss1/Pomezia Dir.Est,44.040022,10.131991,1@FL0471
4,1@FL0502_600,Catagnina/Tinelli Dir.Ovest,44.032957,10.123613,1@FL0502
...,...,...,...,...,...
4617,S08405_1,Roma Trastevere,41.872370,12.466141,S08405
4618,S08406_1,Roma Ostiense,41.872778,12.483893,S08406
4619,S08408_1,Roma Tuscolana,41.879405,12.523398,S08408
4620,S08409_1,Roma Termini,41.900534,12.501878,S08409


In [10]:
# Elimino le shape dei comuni che non sono quelli che voglamo:

comuni_presenti = comuni_list['Comune'].tolist()

comuni_shape = comuni_shape[comuni_shape['COMUNE'].isin(comuni_presenti)]

comuni_shape


,COMUNE,geometry
4273,Bagni di Lucca,"POLYGON ((10.62038 44.11724, 10.62869 44.11755..."
4274,Barga,"POLYGON ((10.52512 44.15624, 10.52723 44.15608..."
4275,Borgo a Mozzano,"POLYGON ((10.50351 44.02197, 10.50522 44.02246..."
4277,Camporgiano,"POLYGON ((10.30806 44.17467, 10.31029 44.17454..."
4279,Careggine,"POLYGON ((10.33471 44.13636, 10.33660 44.13460..."
4280,Castelnuovo di Garfagnana,"POLYGON ((10.38978 44.12633, 10.39669 44.12424..."
4281,Castiglione di Garfagnana,"POLYGON ((10.46705 44.22887, 10.47013 44.22606..."
4282,Coreglia Antelminelli,"POLYGON ((10.62406 44.12037, 10.62869 44.11755..."
4284,Fosciandora,"POLYGON ((10.52461 44.15917, 10.52493 44.15700..."
4285,Gallicano,"MULTIPOLYGON (((10.45096 44.06086, 10.45485 44..."


In [11]:
comuni_shape

,COMUNE,geometry
4273,Bagni di Lucca,"POLYGON ((10.62038 44.11724, 10.62869 44.11755..."
4274,Barga,"POLYGON ((10.52512 44.15624, 10.52723 44.15608..."
4275,Borgo a Mozzano,"POLYGON ((10.50351 44.02197, 10.50522 44.02246..."
4277,Camporgiano,"POLYGON ((10.30806 44.17467, 10.31029 44.17454..."
4279,Careggine,"POLYGON ((10.33471 44.13636, 10.33660 44.13460..."
4280,Castelnuovo di Garfagnana,"POLYGON ((10.38978 44.12633, 10.39669 44.12424..."
4281,Castiglione di Garfagnana,"POLYGON ((10.46705 44.22887, 10.47013 44.22606..."
4282,Coreglia Antelminelli,"POLYGON ((10.62406 44.12037, 10.62869 44.11755..."
4284,Fosciandora,"POLYGON ((10.52461 44.15917, 10.52493 44.15700..."
4285,Gallicano,"MULTIPOLYGON (((10.45096 44.06086, 10.45485 44..."


In [12]:
# Converte il dataframe stops in un GeoDataFrame
stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops['stop_lon'],stops['stop_lat']), crs="EPSG:4326")


In [13]:
# Crea una nuova colonna "poligono" nel dataframe stops per il comune di appartenenza


# Itera attraverso ogni riga nel dataframe "stops"
for index, stop in stops.iterrows():
    stop_point = stop['geometry']
    
    # Itera attraverso ogni riga nel dataframe "comuni_shape"
    for comune_index, comune in comuni_shape.iterrows():
        comune_polygon = comune['geometry']
        
        # Verifica se il punto è all'interno del poligono
        if stop_point.within(comune_polygon):
            # Assegna il nome del comune o altro identificatore al punto corrispondente
            stops.at[index, 'poligono'] = comune['COMUNE']
            break  # Esci dal ciclo interno se il punto è stato assegnato a un poligono

# Rimuovo le righe che hanno nan in poligono

stops = stops[stops['poligono'] != 'nan']

# Rimuovo i possibili duplicati
stops = stops.drop_duplicates(subset='stop_id')

In [14]:
stops

,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,poligono
9,2@900001_600,Fermata Dep. Mologno,44.061137,10.456383,2@900001,POINT (10.45638 44.06114),Barga
11,2@900004_600,Fermata Dep. Mologno,44.061318,10.456306,2@900004,POINT (10.45631 44.06132),Barga
12,2@900005_600,Nebbianella,44.072605,10.458223,2@900005,POINT (10.45822 44.07260),Barga
14,2@C0009_600,Verdi 5,43.842763,10.497259,2@C0009,POINT (10.49726 43.84276),Lucca
18,2@C0061_600,Palatucci,43.847681,10.495580,2@C0061,POINT (10.49558 43.84768),Lucca
...,...,...,...,...,...,...,...
4469,S06323_1,Poggio-Car.-Vag.,44.148485,10.350640,S06323,POINT (10.35064 44.14848),Camporgiano
4470,S06324_1,Camporgiano,44.161474,10.334720,S06324,POINT (10.33472 44.16147),Camporgiano
4471,S06325_1,Piazza Al Serch.,44.182691,10.296700,S06325,POINT (10.29670 44.18269),Piazza al Serchio
4473,S06351_1,Nozzano,43.829912,10.407130,S06351,POINT (10.40713 43.82991),Lucca


In [15]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled
0,4493_32260,08:03:00,08:03:00,6@C6200_600,1,0
1,4493_32260,08:04:00,08:04:00,6@C6202_600,2,1.191
2,4493_32260,08:05:00,08:05:00,6@C6203_600,3,1.482
3,4493_32260,08:29:00,08:29:00,6@C6604_600,4,24.56
4,4493_32260,08:30:00,08:30:00,6@C4923_600,5,24.881
...,...,...,...,...,...,...
325808,4418_299909,17:42:00,17:42:00,S06404_1,1,0.0
325809,4418_299909,17:48:00,17:49:00,S06402_1,2,8.01853
325810,4418_299909,17:57:00,17:58:00,S06400_1,3,14.83468
325811,4418_299909,18:07:00,18:08:00,S06501_1,4,20.90887


In [16]:
stops

,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,poligono
9,2@900001_600,Fermata Dep. Mologno,44.061137,10.456383,2@900001,POINT (10.45638 44.06114),Barga
11,2@900004_600,Fermata Dep. Mologno,44.061318,10.456306,2@900004,POINT (10.45631 44.06132),Barga
12,2@900005_600,Nebbianella,44.072605,10.458223,2@900005,POINT (10.45822 44.07260),Barga
14,2@C0009_600,Verdi 5,43.842763,10.497259,2@C0009,POINT (10.49726 43.84276),Lucca
18,2@C0061_600,Palatucci,43.847681,10.495580,2@C0061,POINT (10.49558 43.84768),Lucca
...,...,...,...,...,...,...,...
4469,S06323_1,Poggio-Car.-Vag.,44.148485,10.350640,S06323,POINT (10.35064 44.14848),Camporgiano
4470,S06324_1,Camporgiano,44.161474,10.334720,S06324,POINT (10.33472 44.16147),Camporgiano
4471,S06325_1,Piazza Al Serch.,44.182691,10.296700,S06325,POINT (10.29670 44.18269),Piazza al Serchio
4473,S06351_1,Nozzano,43.829912,10.407130,S06351,POINT (10.40713 43.82991),Lucca


In [17]:
stop_times = stop_times.merge(stops[['stop_id', 'poligono']], on='stop_id', how='left')
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,poligono
0,4493_32260,08:03:00,08:03:00,6@C6200_600,1,0,Lucca
1,4493_32260,08:04:00,08:04:00,6@C6202_600,2,1.191,Lucca
2,4493_32260,08:05:00,08:05:00,6@C6203_600,3,1.482,Lucca
3,4493_32260,08:29:00,08:29:00,6@C6604_600,4,24.56,NaN
4,4493_32260,08:30:00,08:30:00,6@C4923_600,5,24.881,NaN
...,...,...,...,...,...,...,...
325808,4418_299909,17:42:00,17:42:00,S06404_1,1,0.0,Lucca
325809,4418_299909,17:48:00,17:49:00,S06402_1,2,8.01853,NaN
325810,4418_299909,17:57:00,17:58:00,S06400_1,3,14.83468,NaN
325811,4418_299909,18:07:00,18:08:00,S06501_1,4,20.90887,NaN


In [18]:
# Aggiungi una colonna 'remove_series' al DataFrame stop_times
stop_times['remove_series'] = stop_times.groupby('trip_id')['poligono'].transform(lambda x: any(x.isna()))

# Filtra il DataFrame in base alla colonna 'remove_series'
stop_times = stop_times[~stop_times['remove_series']]

# Elimina la colonna 'remove_series' dal DataFrame risultante
stop_times = stop_times.drop(columns=['remove_series'])

stop_times


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,poligono
2694,4493_3547,18:25:00,18:25:00,2@C4548_600,1,0,Bagni di Lucca
2695,4493_3547,18:26:00,18:26:00,2@C5203_600,2,.393,Bagni di Lucca
2696,4493_3547,18:33:00,18:33:00,2@C5205_600,3,2.336,Bagni di Lucca
2697,4493_3547,18:42:00,18:42:00,2@C5206_600,4,4.893,Bagni di Lucca
2698,4493_3548,07:56:00,07:56:00,2@C4548_600,1,0,Bagni di Lucca
...,...,...,...,...,...,...,...
325798,4418_299905,18:46:00,18:47:00,S06300_1,11,44.93742,Castelnuovo di Garfagnana
325799,4418_299905,18:53:00,18:54:00,S06322_1,12,49.82972,San Romano in Garfagnana
325800,4418_299905,18:56:00,18:57:00,S06323_1,13,51.28115,Camporgiano
325801,4418_299905,19:00:00,19:01:00,S06324_1,14,53.26429,Camporgiano


In [19]:
# Aggiungi una colonna 'poligono_successivo' con il valore della riga successiva in 'poligono'
stop_times['poligono_successivo'] = stop_times.groupby('trip_id')['poligono'].shift(-1)


# Stampa il DataFrame risultante
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,poligono,poligono_successivo
2694,4493_3547,18:25:00,18:25:00,2@C4548_600,1,0,Bagni di Lucca,Bagni di Lucca
2695,4493_3547,18:26:00,18:26:00,2@C5203_600,2,.393,Bagni di Lucca,Bagni di Lucca
2696,4493_3547,18:33:00,18:33:00,2@C5205_600,3,2.336,Bagni di Lucca,Bagni di Lucca
2697,4493_3547,18:42:00,18:42:00,2@C5206_600,4,4.893,Bagni di Lucca,Bagni di Lucca
2698,4493_3548,07:56:00,07:56:00,2@C4548_600,1,0,Bagni di Lucca,Bagni di Lucca
...,...,...,...,...,...,...,...,...
325798,4418_299905,18:46:00,18:47:00,S06300_1,11,44.93742,Castelnuovo di Garfagnana,San Romano in Garfagnana
325799,4418_299905,18:53:00,18:54:00,S06322_1,12,49.82972,San Romano in Garfagnana,Camporgiano
325800,4418_299905,18:56:00,18:57:00,S06323_1,13,51.28115,Camporgiano,Camporgiano
325801,4418_299905,19:00:00,19:01:00,S06324_1,14,53.26429,Camporgiano,Piazza al Serchio


In [20]:
stop_times.dropna(inplace=True)


In [21]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,poligono,poligono_successivo
2694,4493_3547,18:25:00,18:25:00,2@C4548_600,1,0,Bagni di Lucca,Bagni di Lucca
2695,4493_3547,18:26:00,18:26:00,2@C5203_600,2,.393,Bagni di Lucca,Bagni di Lucca
2696,4493_3547,18:33:00,18:33:00,2@C5205_600,3,2.336,Bagni di Lucca,Bagni di Lucca
2697,4493_3547,18:42:00,18:42:00,2@C5206_600,4,4.893,Bagni di Lucca,Bagni di Lucca
2698,4493_3548,07:56:00,07:56:00,2@C4548_600,1,0,Bagni di Lucca,Bagni di Lucca
...,...,...,...,...,...,...,...,...
325797,4418_299905,18:38:00,18:39:00,S06302_1,10,38.10713,Barga,Castelnuovo di Garfagnana
325798,4418_299905,18:46:00,18:47:00,S06300_1,11,44.93742,Castelnuovo di Garfagnana,San Romano in Garfagnana
325799,4418_299905,18:53:00,18:54:00,S06322_1,12,49.82972,San Romano in Garfagnana,Camporgiano
325800,4418_299905,18:56:00,18:57:00,S06323_1,13,51.28115,Camporgiano,Camporgiano


In [22]:
groupByStopTimes = stop_times.groupby(['poligono', 'poligono_successivo'], as_index=False).agg(count=('trip_id','count'))

In [25]:
import pygraphviz as pgv

G = pgv.AGraph(directed=True)
_ = groupByStopTimes.apply(lambda x: G.add_edge(x['poligono'],x['poligono_successivo'],penwidth=1+np.log(x['count']), label = x['count']),axis=1)


In [26]:
G.layout('dot')
G.draw('garfagnana_network.png')

In [27]:
stop_times.to_csv('stopTimesNodes.csv')